In [ ]:
import os
import numpy as np
import torch
from nilearn import plotting
import data
import visualize
from words import make_classifications, Predictions
from data import normalize_fmri_data, unnormalize_fmri_data
from LEM import extract_data_features, predAccuracy
from numpy.linalg import norm


In [ ]:
class argObj:
    def __init__(self, data_dir, parent_submission_dir, subj):
        self.subj = format(subj, '02')
        self.data_dir = os.path.join(data_dir, 'subj' + self.subj)
        self.parent_submission_dir = parent_submission_dir
        self.subject_submission_dir = os.path.join(self.parent_submission_dir,
                                                   'subj' + self.subj)


In [ ]:
platform = 'jupyter_notebook'

device = "cuda:1"
torch.cuda.set_device(device)
# setting up the directories and ARGS
data_dir = '/home/vislab-001/Documents/algonauts_2023_challenge_data/'
parent_submission_dir = '../submission'
subj = 1 # @param ["1", "2", "3", "4", "5", "6", "7", "8"] {type:"raw", allow-input: true}
# args

args = argObj(data_dir, parent_submission_dir, subj)
fmri_dir = os.path.join(args.data_dir, 'training_split', 'training_fmri')
lh_fmri = np.load(os.path.join(fmri_dir, 'lh_training_fmri.npy'))
rh_fmri = np.load(os.path.join(fmri_dir, 'rh_training_fmri.npy'))

words = ['person', 'food', 'kitchenware', 'appliance', 'furniture', 'animal', 'vehicle', 'accessory',
            'electronics', 'sports', 'traffic', 'outdoor', 'home', 'clothing', 'hygiene', 'toy', 'plumbing', 
             'computer', 'fruit', 'vegetable', 'tool']


In [ ]:
print("________ Process Data ________")

# Normalize Data Before Split
lh_fmri, lh_data_min, lh_data_max = normalize_fmri_data(lh_fmri)
print(lh_fmri)
print("- - - - - - - -")
rh_fmri, rh_data_min, rh_data_max = normalize_fmri_data(rh_fmri)
print(rh_fmri)

print('LH training fMRI data shape:')
print(lh_fmri.shape)
print('(Training stimulus images × LH vertices)')

print('\nRH training fMRI data shape:')
print(rh_fmri.shape)
print('(Training stimulus images × RH vertices)')

train_img_dir = os.path.join(args.data_dir, 'training_split', 'training_images')
test_img_dir = os.path.join(args.data_dir, 'test_split', 'test_images')
train_img_list = os.listdir(train_img_dir)
train_img_list.sort()
# train_img_list = train_img_list[:2500]
test_img_list = os.listdir(test_img_dir)
test_img_list.sort()

print('\nTraining images: ' + str(len(train_img_list)))
print('\nTest images: ' + str(len(test_img_list)))
train_img_file = train_img_list[0]
print('\nTraining image file name: ' + train_img_file)
print('\n73k NSD images ID: ' + train_img_file[-9:-4])

In [ ]:
print("________ Split Data ________")

idxs_train, idxs_val, idxs_test = data.splitdata(train_img_list, test_img_list, train_img_dir)
lh_fmri_train = lh_fmri[idxs_train]
rh_fmri_train = rh_fmri[idxs_train]
lh_fmri_val = lh_fmri[idxs_val]
rh_fmri_val = rh_fmri[idxs_val]
lh_fmri_test = lh_fmri[idxs_test]
rh_fmri_test = rh_fmri[idxs_test]

In [ ]:
print("________ Make Lists ________")

train_images = data.makeList(train_img_dir, train_img_list, idxs_train)
val_images = data.makeList(train_img_dir, train_img_list, idxs_val)
test_images = data.makeList(test_img_dir, test_img_list, idxs_test)
torch.cuda.empty_cache()

In [7]:
print("________ Make Classifications ________")
lh_classifications = make_classifications(train_images, idxs_train, device)
rh_classifications = lh_classifications
lh_classifications_val = make_classifications(val_images, idxs_val, device)
rh_classifications_val = lh_classifications_val
lh_classifications_test = make_classifications(test_images, idxs_test, device)
rh_classifications_test = lh_classifications_test

torch.cuda.empty_cache()

0: 640x640 2 persons, 1 surfboard, 1: 640x640 1 cat, 1 laptop, 1 mouse, 1 book, 2: 640x640 5 persons, 2 skiss, 3: 640x640 1 person, 2 benchs, 1 bottle, 4: 640x640 2 giraffes, 5: 640x640 2 persons, 1 tennis racket, 6: 640x640 1 laptop, 1 mouse, 7: 640x640 1 person, 6 wine glasss, 3 cups, 1 bowl, 2 chairs, 1 cell phone, 8: 640x640 1 clock, 9: 640x640 4 sheeps, 10: 640x640 1 person, 1 bench, 11: 640x640 1 person, 8 bottles, 1 refrigerator, 12: 640x640 7 persons, 3 bicycles, 1 car, 1 umbrella, 13: 640x640 1 boat, 1 kite, 1 clock, 14: 640x640 10 persons, 6 motorcycles, 15: 640x640 1 car, 1 parking meter, 16: 640x640 1 person, 2 kites, 17: 640x640 3 persons, 18: 640x640 1 truck, 19: 640x640 3 cats, 1 bottle, 20: 640x640 5 persons, 1 cup, 1 potted plant, 1 tv, 21: 640x640 1 car, 1 stop sign, 22: 640x640 1 person, 1 surfboard, 23: 640x640 2 cats, 24: 640x640 2 elephants, 25: 640x640 4 persons, 3 cars, 1 bench, 1 umbrella, 26: 640x640 1 person, 2 ties, 1 couch, 1 bed, 1 remote, 1 clock, 27: 640

In [8]:
import torch
from numpy import average
import numpy as np
from sklearn.decomposition import IncrementalPCA
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor
from tqdm import tqdm
from scipy.stats import pearsonr as corr
from torchvision.models import vgg19, efficientnet_v2_s, inception_v3, resnet50, resnet101, wide_resnet50_2, wide_resnet101_2
from ultralytics import YOLO

class EfficientNetThroughLastConvExtractor(torch.nn.Module):
    def __init__(self):
        super(EfficientNetThroughLastConvExtractor, self).__init__()
        self.efficientNet = efficientnet_v2_s()
        self.features = self.efficientNet.features[:-1]
        self.finalConvLayer = list(self.efficientNet.features[-1].children())[0]

    def forward(self, img):
        features = self.features(img)
        features = self.finalConvLayer(features)
        return features

    def __str__(self):
        return ""
    
    def __repr__(self):
        return ""


class EfficientNetFeatureExtractor(torch.nn.Module):
    def __init__(self):
        super(EfficientNetFeatureExtractor, self).__init__()
        self.efficientNet = efficientnet_v2_s()

    def forward(self, img):
        features = self.efficientNet.features(img)
        return features

    def __str__(self):
        return ""
    
    def __repr__(self):
        return ""

class InceptionEXtractor(torch.nn.Module):
    def __init__(self):
        super(InceptionEXtractor, self).__init__()
        self.inception = inception_v3(weights="DEFAULT")

    def forward(self, x):
        # N x 3 x 299 x 299
        x = self.inception.Conv2d_1a_3x3(x)
        # N x 32 x 149 x 149
        x = self.inception.Conv2d_2a_3x3(x)
        # N x 32 x 147 x 147
        x = self.inception.Conv2d_2b_3x3(x)
        # N x 64 x 147 x 147
        x = self.inception.maxpool1(x)
        # N x 64 x 73 x 73
        x = self.inception.Conv2d_3b_1x1(x)
        # N x 80 x 73 x 73
        x = self.inception.Conv2d_4a_3x3(x)
        # N x 192 x 71 x 71
        x = self.inception.maxpool2(x)
        # N x 192 x 35 x 35
        x = self.inception.Mixed_5b(x)
        # N x 256 x 35 x 35
        x = self.inception.Mixed_5c(x)
        # N x 288 x 35 x 35
        x = self.inception.Mixed_5d(x)
        # N x 288 x 35 x 35
        x = self.inception.Mixed_6a(x)
        # N x 768 x 17 x 17
        x = self.inception.Mixed_6b(x)
        # N x 768 x 17 x 17
        x = self.inception.Mixed_6c(x)
        # N x 768 x 17 x 17
        x = self.inception.Mixed_6d(x)
        # N x 768 x 17 x 17
        x = self.inception.Mixed_6e(x)
        # N x 768 x 17 x 17
        x = self.inception.Mixed_7a(x)
        # N x 1280 x 8 x 8
        x = self.inception.Mixed_7b(x)
        # N x 2048 x 8 x 8
        x = self.inception.Mixed_7c(x)
        # N x 2048 x 8 x 8
        # # Adaptive average pooling
        # x = self.inception.avgpool(x)

        return x

class Resnet50Extractor(torch.nn.Module):
    def __init__(self):
        super(Resnet50Extractor, self).__init__()
        self.resnet = resnet50(weights="DEFAULT")

    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)

        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)

        return x

class Resnet101Extractor(torch.nn.Module):
    def __init__(self):
        super(Resnet101Extractor, self).__init__()
        self.resnet = resnet101(weights="DEFAULT")

    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)

        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)

        return x

class WideResnet502(torch.nn.Module):
    def __init__(self):
        super(WideResnet502, self).__init__()
        self.resnet = wide_resnet50_2(weights="DEFAULT")

    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)

        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)

        return x

class WideResnet1012(torch.nn.Module):
    def __init__(self):
        super(WideResnet1012, self).__init__()
        self.resnet = wide_resnet101_2(weights="DEFAULT")

    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)

        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)

        return x

class YoloFeatureExtractor(torch.nn.Module):
    def __init__(self):
        super(YoloFeatureExtractor, self).__init__()
        yolo = YOLO("yolov8n.pt").to("cuda:1")
        self.featureExtractor = list(yolo.model.model.children())[:10]
        self.featureExtractor = torch.nn.Sequential(*self.featureExtractor)

    def forward(self, x):
        return self.featureExtractor(x)


def extract_data_features(train_imgs_dataloader, val_imgs_dataloader, test_imgs_dataloader, batch_size, device = "cuda:1"):
    # vgg = vgg19(weights="DEFAULT").to(device)
    # vggConvFeatures = vgg.features[:35]
    # model_layer = "avgpool"

    feature_extractor = Resnet50Extractor()#create_feature_extractor(vgg, return_nodes=[model_layer])
    pca = fit_pca(feature_extractor, train_imgs_dataloader, batch_size, device)

    features_train = extract_features(feature_extractor, train_imgs_dataloader, pca, device)
    features_val = extract_features(feature_extractor, val_imgs_dataloader, pca, device)
    features_test = extract_features(feature_extractor, test_imgs_dataloader, pca, device)

    print('\nTraining images features:')
    print(features_train.shape)
    print('(Training stimulus images × PCA features)')

    print('\nValidation images features:')
    print(features_val.shape)
    print('(Validation stimulus images × PCA features)')

    print('\nTest images features:')
    print(features_test.shape)
    print('(Test stimulus images × PCA features)')

    del pca
    return features_train, features_val, features_test



def predAccuracy(lh_fmri_val_pred, lh_fmri_val, rh_fmri_val_pred, rh_fmri_val):
    print("Start PredAccuracy")
    print("\npredicted\n", lh_fmri_val_pred, "\nactual\n", lh_fmri_val)

    # Empty correlation array of shape: (LH vertices)
    lh_correlation = np.zeros(lh_fmri_val_pred.shape[1])

    # Correlate each predicted LH vertex with the corresponding ground truth vertex
    for v in tqdm(range(lh_fmri_val_pred.shape[1])):
        lh_correlation[v] = corr(lh_fmri_val_pred[:, v], lh_fmri_val[:, v])[0]

    # Empty correlation array of shape: (RH vertices)
    rh_correlation = np.zeros(rh_fmri_val_pred.shape[1])

    # Correlate each predicted RH vertex with the corresponding ground truth vertex
    for v in tqdm(range(rh_fmri_val_pred.shape[1])):
        rh_correlation[v] = corr(rh_fmri_val_pred[:, v], rh_fmri_val[:, v])[0]
        
    print('average lh ', average(lh_correlation) * 100, 'average rh ', average(rh_correlation) * 100)
    return lh_correlation, rh_correlation

def extract_features(feature_extractor, dataloader, pca, device):
    features = []
    shape = 0
    for _, d in tqdm(enumerate(dataloader), total=len(dataloader), desc="Extracting Features"):
        # Send to tensor to device
        d = d.to(device)
        # ...
        # Extract features
        ft = feature_extractor(d)
        shape = ft.shape
        # Flatten the features
        ft = torch.flatten(ft, 1)
        # Apply PCA transform
        ft = pca.transform(ft.cpu().detach().numpy())
        # ft = pca.transform(ft.cuda().detach().numpy())
        features.append(ft)
        val = np.vstack(features)
        # print(val)
    print(shape)
    return np.vstack(features)


def fit_pca(feature_extractor, dataloader, batch_size, device):
    # Define PCA parameters
    pca = IncrementalPCA(batch_size=batch_size)
    # Fit PCA to batch
    for _, d in tqdm(enumerate(dataloader), total=len(dataloader), desc="PCA"):
        # Send to tensor to device
        d = d.to(device)
        # Extract features
        ft = feature_extractor(d)
        # Flatten the features
        ft = torch.flatten(ft, 1)
        # Fit PCA to batch
        pca.partial_fit(ft.detach().cpu().numpy())
    return pca

In [9]:
print("________ Extract Image Features ________")

train_imgs_dataloader, val_imgs_dataloader, test_imgs_dataloader = (
    data.transformData(train_img_dir, test_img_dir, idxs_train, idxs_val, idxs_test, 64))


________ Extract Image Features ________


In [10]:
features_train, features_val, features_test = (
    extract_data_features(train_imgs_dataloader, val_imgs_dataloader, test_imgs_dataloader, 32, device))

Extracting Features: 100%|██████████| 116/116 [01:25<00:00,  1.35it/s]


torch.Size([20, 512, 14, 14])


Extracting Features: 100%|██████████| 39/39 [00:30<00:00,  1.26it/s]


torch.Size([29, 512, 14, 14])


Extracting Features: 100%|██████████| 3/3 [00:02<00:00,  1.43it/s]

torch.Size([31, 512, 14, 14])

Training images features:
(7380, 64)
(Training stimulus images × PCA features)

Validation images features:
(2461, 64)
(Validation stimulus images × PCA features)

Test images features:
(159, 64)
(Test stimulus images × PCA features)


In [11]:
features_train.shape

(7380, 64)

In [12]:
print("________ Organize Input________")

dftrainL, dftrainFL = data.organize_input(lh_classifications, features_train, lh_fmri_train)
dftrainR, dftrainFR = data.organize_input(rh_classifications, features_train, rh_fmri_train)

dfvalL, dfvalFL = data.organize_input(lh_classifications_val, features_val, lh_fmri_val)
dfvalR, dfvalFR = data.organize_input(rh_classifications_val, features_val, rh_fmri_val)

# dftestL, dftestFL = data.organize_input(lh_classifications_test, features_test, lh_fmri_test)
# dftestR, dftestFR = data.organize_input(rh_classifications_test, features_test, rh_fmri_test)

________ Organize Input________


In [13]:
print("________ Predictions Validation ________")
lh_fmri_val_pred = Predictions(dftrainL, dftrainFL, dfvalL, dfvalFL)
rh_fmri_val_pred = Predictions(dftrainR, dftrainFR, dfvalR, dfvalFR)

________ Predictions Validation ________
[array([     0.5852,     0.39548,     0.49428, ...,     0.43907,     0.42352,     0.52506], dtype=float32), array([    0.43593,     0.67775,     0.51437, ...,       0.813,     0.79775,     0.59968], dtype=float32), array([    0.52045,     0.36821,     0.52356, ...,     0.53712,     0.57273,     0.55908], dtype=float32), array([    0.14485,     0.43538,     0.39298, ...,     0.57542,     0.56758,     0.33398], dtype=float32), array([    0.47129,     0.56882,       0.804, ...,     0.52991,     0.59896,     0.38291], dtype=float32), array([    0.25739,     0.52559,     0.41252, ...,     0.45486,     0.33075,     0.38508], dtype=float32), array([    0.64936,     0.59469,     0.75737, ...,     0.50304,     0.53521,     0.47604], dtype=float32), array([    0.52248,      0.6082,     0.26216, ...,      0.4269,     0.37863,     0.60301], dtype=float32), array([    0.17299,     0.50198,     0.47666, ...,      0.7473,     0.72818,     0.67651], dtype=float

In [14]:
# print("________ Predictions Test________")
# lh_fmri_test_pred = Predictions(dftrainL, dftrainFL, dftestL, dftestFL)
# rh_fmri_test_pred = Predictions(dftrainR, dftrainFR, dftestR, dftestFR)

In [15]:
print("________ Unnormalize Data ________")

lh_fmri_val_pred = unnormalize_fmri_data(lh_fmri_val_pred, lh_data_min, lh_data_max)
rh_fmri_val_pred = unnormalize_fmri_data(rh_fmri_val_pred, rh_data_min, rh_data_max)
# lh_fmri_test_pred = unnormalize_fmri_data(lh_fmri_test_pred, lh_data_min, lh_data_max)
# rh_fmri_test_pred = unnormalize_fmri_data(rh_fmri_test_pred, rh_data_min, rh_data_max)


________ Unnormalize Data ________


In [16]:
print("________ Re-Load Data ________")
lh_fmri = np.load(os.path.join(fmri_dir, 'lh_training_fmri.npy'))
rh_fmri = np.load(os.path.join(fmri_dir, 'rh_training_fmri.npy'))

lh_fmri_val = lh_fmri[idxs_val]
rh_fmri_val = rh_fmri[idxs_val]

lh_fmri_test = lh_fmri[idxs_train]
rh_fmri_test = rh_fmri[idxs_train]
print(lh_fmri_test)


________ Re-Load Data ________
[[   -0.62598    -0.18787     0.17061 ...   -0.071224    -0.11506     -1.3529]
 [   -0.75055    -0.86018     -1.1247 ...    -0.92085    -0.76416    -0.36854]
 [     0.3336      1.0938      1.1228 ...     0.51576     0.53022      1.1962]
 ...
 [    0.72472    0.054428    -0.13178 ...    -0.99088     -1.2171    -0.30849]
 [   -0.21365       0.458    -0.29252 ...    -0.58329    -0.65519    0.045681]
 [   -0.55378    -0.52293    -0.25691 ...     -0.6328    -0.32734    -0.39385]]


In [17]:
data.analyze_results(lh_fmri_val, lh_fmri_val_pred)

[[    0.40555    -0.54157   -0.048368 ...    -0.32394    -0.40157     0.10533]
 [   -0.33966     0.86755    0.051938 ...      1.5427      1.4666     0.47781]
 [   0.082277    -0.67769    0.097822 ...      0.1655     0.34329     0.27515]
 ...
 [    0.80241     0.39293     0.40859 ...     0.54396     0.40171     0.77187]
 [    0.13767    -0.36039    -0.32033 ...    -0.65734    -0.79607     -0.8407]
 [    0.63283    -0.63886    0.080614 ...     0.70695      0.6085      1.2763]] 
 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 [[    0.16294     0.26304     0.48939 ...     0.29823     0.29545    0.075047]
 [  -0.037273    -0.11039    -0.05003 ...    0.066559    0.072906     0.22019]
 [   -0.06324     0.42477    0.017681 ...     0.28172      0.2838    0.075097]
 ...
 [   0.047203    -0.11273    -0.59139 ...    -0.22642    -0.19168  0.00078134]
 [   -0.26676    -0.33278    -0.46426 ...   -0.075657   -0.098717     0.09493]
 [    0.11628     -0.1857    -0.35649 ...   -0.091865   -0.085902    0.027951]]
Mean Sq

In [18]:
data.analyze_results(rh_fmri_val, rh_fmri_val_pred)

[[    0.39951    0.060797    -0.16728 ...    -0.69501    -0.48838    -0.52206]
 [    -1.0012      1.3242     0.22229 ...       1.488      1.6151      1.7359]
 [   -0.51003     0.87209     -2.0209 ...     0.17798   -0.022853    -0.50177]
 ...
 [   -0.16802     0.28208      1.0006 ...     0.52578     0.87302     0.90553]
 [  -0.082318     0.47183     0.31499 ...     0.10405   -0.052576   -0.087478]
 [   -0.14104    -0.38522     0.68105 ...      0.2289     0.10505    -0.17628]] 
 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 [[     0.2271     0.34973     0.35813 ...    -0.34563    -0.26648    -0.20067]
 [   -0.17932     0.24641    -0.12419 ...     0.42334     0.35806     0.30426]
 [    0.12019    0.099846    -0.17838 ...    -0.28467    -0.23052    -0.21344]
 ...
 [   -0.18702   -0.077282     0.96647 ...     -0.1549    -0.23128    -0.26282]
 [   -0.17463   0.0097609   -0.091646 ...     0.31812     0.27939     0.25459]
 [  -0.021213     0.19513     0.81707 ...     0.12027    0.091685    0.098538]]
Mean Sq

In [ ]:
print("________ Predictions Test________")
# lh_fmri_test_pred = Predictions(dftrainL, dftrainFL, dftestL, dftestFL)
# rh_fmri_test_pred = Predictions(dftrainR, dftrainFR, dftestR, dftestFR)

In [19]:
print("________ Prediction Accuracy ________")

lh_correlation, rh_correlation = predAccuracy(lh_fmri_val_pred, lh_fmri_val, rh_fmri_val_pred, rh_fmri_val)


________ Prediction Accuracy ________
Start PredAccuracy

predicted
 [[    0.16294     0.26304     0.48939 ...     0.29823     0.29545    0.075047]
 [  -0.037273    -0.11039    -0.05003 ...    0.066559    0.072906     0.22019]
 [   -0.06324     0.42477    0.017681 ...     0.28172      0.2838    0.075097]
 ...
 [   0.047203    -0.11273    -0.59139 ...    -0.22642    -0.19168  0.00078134]
 [   -0.26676    -0.33278    -0.46426 ...   -0.075657   -0.098717     0.09493]
 [    0.11628     -0.1857    -0.35649 ...   -0.091865   -0.085902    0.027951]] 
actual
 [[    0.40555    -0.54157   -0.048368 ...    -0.32394    -0.40157     0.10533]
 [   -0.33966     0.86755    0.051938 ...      1.5427      1.4666     0.47781]
 [   0.082277    -0.67769    0.097822 ...      0.1655     0.34329     0.27515]
 ...
 [    0.80241     0.39293     0.40859 ...     0.54396     0.40171     0.77187]
 [    0.13767    -0.36039    -0.32033 ...    -0.65734    -0.79607     -0.8407]
 [    0.63283    -0.63886    0.080614 ... 

100%|██████████| 20544/20544 [00:02<00:00, 6932.21it/s]

average lh  37.80140895037795 average rh  38.329446274852174


In [ ]:
print("________ Visualize Each Class ________")

length = len(words)
for clss in range(length):
    print(clss, words[clss])
    avg_lh_pred = []
    avg_lh_real = []
    avg_rh_pred = []
    avg_rh_real = []
    for i in range(len(lh_classifications_val)):

        if lh_classifications_val[i][1] == clss:
            avg_lh_pred.append(lh_fmri_val_pred[i])
            avg_lh_real.append(lh_fmri_val[i])

        if rh_classifications[i][1] == clss:
            avg_rh_pred.append(rh_fmri_val_pred[i])
            avg_rh_real.append(lh_fmri_val[i])

    if(len(avg_lh_pred)  == 0):
        pass
    else:
        lh = np.mean(avg_lh_pred, axis=0)
        rh = np.mean(avg_rh_pred, axis=0)

        #print("MEAN PRED LH:\n", lh)
        #print("MEAN PRED RH:\n", rh)
        visualize.plot_predictions(args, lh, rh)
        lh2 = np.mean(avg_lh_real, axis=0)
        rh2 = np.mean(avg_rh_real, axis=0)

        #print("MEAN REAL LH:\n", lh2)
        #print("MEAN REAL RH:\n", rh2)
        visualize.plot_predictions(args, lh2, rh2)
        plotting.show()

        corr = np.corrcoef(avg_lh_pred, avg_lh_real)
        print(len(avg_lh_pred))
        print("Corre ", np.mean(corr))
        cosine = np.dot(lh,lh2)/(norm(lh)*norm(lh2))
        print("Cosine Similarity:", cosine)
        torch.cuda.empty_cache()

print("________ END ________")

In [ ]:
#minus the graphs to see what happens
print("________ Visualize Each Class ________")

length = len(words)
for clss in range(length):
    print(clss, words[clss])
    avg_lh_pred = []
    avg_lh_real = []
    avg_rh_pred = []
    avg_rh_real = []
    for i in range(len(lh_classifications_val)):

        if lh_classifications_val[i][1] == clss:
            avg_lh_pred.append(lh_fmri_val_pred[i])
            avg_lh_real.append(lh_fmri_val[i])

        if rh_classifications[i][1] == clss:
            avg_rh_pred.append(rh_fmri_val_pred[i])
            avg_rh_real.append(rh_fmri_val[i])

    if(len(avg_lh_pred)  == 0):
        pass
    else:
        lh = np.mean(avg_lh_pred, axis=0)
        rh = np.mean(avg_rh_pred, axis=0)

        #print("MEAN PRED LH:\n", lh)
        #print("MEAN PRED RH:\n", rh)
        lh2 = np.mean(avg_lh_real, axis=0)
        rh2 = np.mean(avg_rh_real, axis=0)

        lh = lh2 - lh
        visualize.plot_predictions(args, lh, rh)


        #print("MEAN REAL LH:\n", lh2)
        #print("MEAN REAL RH:\n", rh2)
        visualize.plot_predictions(args, lh2, rh2)
        plotting.show()

        corr = np.corrcoef(avg_lh_pred, avg_lh_real)
        print(len(avg_lh_pred))
        print("Corre ", np.mean(corr))
        cosine = np.dot(lh,lh2)/(norm(lh)*norm(lh2))
        print("Cosine Similarity:", cosine)
        torch.cuda.empty_cache()

print("________ END ________")

In [ ]:
#minus the graphs to see what happens but trying another way
print("________ Visualize Each Class ________")


length = len(words)
for clss in range(length):
    print(clss, words[clss])
    avg_lh_pred = []
    avg_lh_real = []
    avg_rh_pred = []
    avg_rh_real = []
    for i in range(len(lh_classifications_val)):

        if lh_classifications_val[i][1] == clss:
            avg_lh_pred.append(testLhVal[i])
            avg_lh_real.append(lh_fmri_val[i])

        if rh_classifications[i][1] == clss:
            avg_rh_pred.append(testRhVal[i])
            avg_rh_real.append(rh_fmri_val[i])

    if(len(avg_lh_pred)  == 0):
        pass
    else:
        lh = np.mean(avg_lh_pred, axis=0)
        rh = np.mean(avg_rh_pred, axis=0)

        #print("MEAN PRED LH:\n", lh)
        #print("MEAN PRED RH:\n", rh)
        visualize.plot_predictions(args, lh, rh)
        lh2 = np.mean(avg_lh_real, axis=0)
        rh2 = np.mean(avg_rh_real, axis=0)

        lh = lh2 - lh
        rh = rh2 - rh

        #print("MEAN REAL LH:\n", lh2)
        #print("MEAN REAL RH:\n", rh2)
        visualize.plot_predictions(args, lh2, rh2)
        plotting.show()

        corr = np.corrcoef(avg_lh_pred, avg_lh_real)
        print(len(avg_lh_pred))
        print("Corre ", np.mean(corr))
        cosine = np.dot(lh,lh2)/(norm(lh)*norm(lh2))
        print("Cosine Similarity:", cosine)
        torch.cuda.empty_cache()

print("________ END ________")

In [ ]:
print("________ Visualize Each Class ________")

length = len(words)
for clss in range(length):
    print(clss, words[clss])
    avg_lh_pred = []
    avg_lh_real = []
    avg_rh_pred = []
    avg_rh_real = []
    for i in range(len(lh_classifications_test)):

        if lh_classifications_test[i][1] == clss:
            avg_lh_pred.append(lh_fmri_test_pred[i])
            avg_lh_real.append(lh_fmri_test[i])

        if rh_classifications[i][1] == clss:
            avg_rh_pred.append(rh_fmri_test_pred[i])
            avg_rh_real.append(lh_fmri_test[i])

    if(len(avg_lh_pred)  == 0):
        pass
    else:
        lh = np.mean(avg_lh_pred, axis=0)
        rh = np.mean(avg_rh_pred, axis=0)
        lh = np.sign(lh)
        rh = np.sign(rh)
        print("MEAN PRED LH:\n", lh)
        print("MEAN PRED RH:\n", rh)
        visualize.plot_predictions(args, lh, rh)
        lh2 = np.mean(avg_lh_real, axis=0)
        rh2 = np.mean(avg_rh_real, axis=0)
        lh2 = np.sign(lh2)
        rh2 = np.sign(rh2)

        print("MEAN REAL LH:\n", lh2)
        print("MEAN REAL RH:\n", rh2)
        visualize.plot_predictions(args, lh2, rh2)
        plotting.show()

        corr = np.corrcoef(avg_lh_pred, avg_lh_real)
        print(len(avg_lh_pred))
        print("Corre ", np.mean(corr))
        cosine = np.dot(lh,lh2)/(norm(lh)*norm(lh2))
        print("Cosine Similarity:", cosine)
        torch.cuda.empty_cache()

print("________ END ________")